# Plumbing
1. Download the phrase similarity dataset from http://homepages.inf.ed.ac.uk/mlap/resources/index.html, save as `phrase_similarities.txt`
2. Download the EasyCCG parser from http://homepages.inf.ed.ac.uk/s1049478/easyccg.html, unpack the package (you should get a catalog like `easyccg-0.2`). From the same page, download the regular pretrained model (`model.tar.gz`). Unpack the model to the parser's catalog.

# Getting the British National Corpus & the word list

We will parse BNC XML files with lxml. NLTK technically has a dedicated parser for BNC, which is extremely slow in the lazy mode, and in the non-lazy mode it is very slow and also consumes >8GB of memory.

In [1]:
bnc_path = 'BNC/Texts/'
from os.path import exists

def bnc_files_iter():
    top_level = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'J', 'K']
    symbols = top_level + ['L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'W', 'V', 'X', 'Y', 'Z',
                           '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    for top in top_level:
        top_path = bnc_path + '/' + top
        if not exists(top_path):
            continue
        for symbol2 in symbols:
            path2 = top_path + '/' + top + symbol2
            if not exists(path2):
                continue
            for symbol3 in symbols:
                current_path = path2 + '/' + top + symbol2 + symbol3 + '.xml'
                if not exists(current_path):
                    continue
                yield open(current_path)

In [2]:
from lxml import etree

In [3]:
unique_words = set()

for bnc_file in bnc_files_iter():
    file_tree = etree.parse(bnc_file)
    for element in file_tree.iter():
        if (element.tag == 'w' or element.tag == 'c') and element.text:
            unique_words.add(element.text.strip())
    bnc_file.close()
    
unique_words = list(unique_words)
print(unique_words[:10])

['', 'windflow', 'Chauliac', 'Fujisankei', 'P(A)', 'race-horses', 'Tigrayans', 'RINK', 'Targeted', 'modicum']


In [4]:
unique_count = len(unique_words)
print(unique_count)

705241


In [20]:
# try stemming just for the embedding?
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()
stemmed_words = [stemmer.stem(word) for word in unique_words]
stemmed_words = list(set(stemmed_words))
print(len(stemmed_words))

497225


# Getting CCG parse trees for BNC

In [5]:
# we will run the underlying parser as a subprocess, and intercept its outputs from within Python
from subprocess import Popen, PIPE, STDOUT
p = Popen(['java', '-jar', 'easyccg-0.2/easyccg.jar', '--model', 'easyccg-0.2/model'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
# .encode() gives bytes instead of str, as .communicate() requires. We get a pair (stdout, stderr):
(parse, err) = p.communicate(input='The cat chases a ball of yarn.\n'.encode())
print(parse, '\n', err)
p.terminate()

b'ID=1\n(<T S[dcl] 1 2> (<T NP[nb] 0 2> (<L NP[nb]/N POS POS The NP[nb]/N>) (<L N POS POS cat N>) ) (<T S[dcl]\\NP 0 2> (<L (S[dcl]\\NP)/NP POS POS chases (S[dcl]\\NP)/NP>) (<T NP[nb] 0 2> (<T NP[nb] 0 2> (<L NP[nb]/N POS POS a NP[nb]/N>) (<L N POS POS ball N>) ) (<T NP\\NP 0 2> (<L (NP\\NP)/NP POS POS of (NP\\NP)/NP>) (<T NP 0 1> (<L N POS POS yarn. N>) ) ) ) ) ) \n' 
 b'Loading model...\nModel loaded, ready to parse.\n'


Let's see how NLTK can handle parse trees.

In [6]:
# some string cleanup
def clean_parse_output(parse_output):
    # (remember we have to deal with the parse returned as bytes, not a Unicode string)
    lines = str(parse_output).split('\\n')
    if len(lines) > 1:
        return lines[1] # the second line contains the parse itself
    else:
        return lines[0]

from nltk.tree import ParentedTree
tree = ParentedTree.fromstring(clean_parse_output(parse))
print(tree)

(<T
  S[dcl]
  1
  2>
  (<T
    NP[nb]
    0
    2>
    (<L NP[nb]/N POS POS The NP[nb]/N>)
    (<L N POS POS cat N>))
  (<T
    S[dcl]\\NP
    0
    2>
    (<L (S[dcl]\\NP ) /NP POS POS chases (S[dcl]\\NP ) /NP>)
    (<T
      NP[nb]
      0
      2>
      (<T
        NP[nb]
        0
        2>
        (<L NP[nb]/N POS POS a NP[nb]/N>)
        (<L N POS POS ball N>))
      (<T
        NP\\NP
        0
        2>
        (<L (NP\\NP ) /NP POS POS of (NP\\NP ) /NP>)
        (<T NP 0 1> (<L N POS POS yarn. N>))))))


It's not very pretty, because NLTK decides to print a newline instead of space inside the less/more than signs. In each (parenthesized expression), the first item (head) is the category of node, and two next items are its child nodes.

In [7]:
def traverse(tree):
    for node in tree:
        return

In [8]:
trees = []
p = Popen(['java', '-jar', 'easyccg-0.2/easyccg.jar', '--model', 'easyccg-0.2/model'], stdout=PIPE, stdin=PIPE, stderr=PIPE)

for bnc_file in bnc_files_iter():
    file_tree = etree.parse(bnc_file)
    for element in file_tree.iter():
        if element.tag == 's':
            sentence = ''
            for nested_element in element.iter():
                if (nested_element.tag == 'w' or nested_element.tag == 'c') and nested_element.text:
                    sentence += ' ' + nested_element.text
            parse_output = p.communicate(input=sentence.encode())[0]
            p.terminate()
            trees.append(clean_parse_output(parse_output))
            p = Popen(['java', '-jar', 'easyccg-0.2/easyccg.jar', '--model', 'easyccg-0.2/model'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
    bnc_file.close()
    if len(trees) >= 3:
        break
   
print(trees[:3])
p.terminate()

["(<T S[dcl] 1 2> (<T NP 1 2> (<L LRB POS POS \\xe2\\x80\\x98 LRB>) (<T NP 0 1> (<L N POS POS Arrest N>) ) ) (<T S[dcl]\\\\NP 0 2> (<T S[dcl]\\\\NP 0 2> (<T (S[dcl]\\\\NP)/PP 0 2> (<L (S[dcl]\\\\NP)/PP POS POS warrant (S[dcl]\\\\NP)/PP>) (<L (S\\\\NP)\\\\(S\\\\NP) POS POS out (S\\\\NP)\\\\(S\\\\NP)>) ) (<T (S[X]\\\\NP)\\\\((S[X]\\\\NP)/PP) 0 1> (<T PP 0 2> (<L PP/NP POS POS for PP/NP>) (<T NP 0 1> (<T N 1 2> (<L N/N POS POS Clowes N/N>) (<T N 1 2> (<L N/N POS POS \\xe2\\x80\\x99 N/N>) (<T N 1 2> (<L N/N POS POS partner N/N>) (<L N POS POS years N>) ) ) ) ) ) ) ) (<T (S\\\\NP)\\\\(S\\\\NP) 0 2> (<L ((S\\\\NP)\\\\(S\\\\NP))/NP POS POS before ((S\\\\NP)\\\\(S\\\\NP))/NP>) (<T NP 0 2> (<T NP 0 1> (<L N POS POS collapse' N>) ) (<L . POS POS . .>) ) ) ) ) ", '(<T NP 1 2> (<L NP/NP POS POS By NP/NP>) (<T NP 0 1> (<T N 1 2> (<L N/N POS POS Daniel N/N>) (<L N POS POS John N>) ) ) ) ', '(<T S[dcl] 1 2> (<T NP 0 2> (<T NP 0 1> (<L N POS POS AWARRANT N>) ) (<T NP\\\\NP 0 2> (<L (NP\\\\NP)/NP POS P

## Learning word embeddings

Our embedding procedure will be based on this Tensorflow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec).

In [8]:
# Consistently map each unique word to a integer.
word_map = {word: index for index, word in enumerate(unique_words) }

In [9]:
# Collect all words from the corpus, as their indices in the word map.
corpus_words = []

for bnc_file in bnc_files_iter():
    file_tree = etree.parse(bnc_file)
    for element in file_tree.iter():
        if (element.tag == 'w' or element.tag == 'c') and element.text:
            corpus_words.append(word_map[element.text.strip()])
    bnc_file.close()

Generate batches of pairs (context word, target word). For simplicity, we hardcode the window size (2) and number of examples in window.

In [10]:
import numpy as np

In [11]:
from random import randint

num_samples = 4

def skipgrams_batch(batch_size):
    assert batch_size % num_samples == 0
    windows_n = batch_size // num_samples
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    for i in range(windows_n):
        target = randint(2, len(corpus_words)-3)
        for j in range(num_samples):
            labels[i*num_samples+j][0] = corpus_words[target]
        batch[i*num_samples] = corpus_words[target-2]
        batch[i*num_samples+1] = corpus_words[target-1]
        batch[i*num_samples+2] = corpus_words[target+1]
        batch[i*num_samples+3] = corpus_words[target+2]
        
    return batch, labels

print(skipgrams_batch(12))

(array([598819, 501722, 300657, 182194, 501722, 491511, 501722, 491511,
       245888,  28561, 405642, 304179], dtype=int32), array([[303895],
       [303895],
       [303895],
       [303895],
       [258481],
       [258481],
       [258481],
       [258481],
       [265205],
       [265205],
       [265205],
       [265205]], dtype=int32))


In [12]:
import tensorflow as tf
import math

In [13]:
vocabulary_size = len(unique_words)
embedding_size = 70

# Model parameters.
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                                              stddev=1.0 / math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

In [14]:
batch_size = 128

# The computation graph.
inputs = tf.placeholder(tf.int32, shape=[batch_size])
labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
embedding_layer = tf.nn.embedding_lookup(embeddings, inputs)

# Note that word2vec has no "real" hidden layers apart from the embedding.

# Number of random words to sample apart from the true target; the model should learn to
# assign low probability to them given the context.
negative_samples_n = 64

loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                     biases=nce_biases,
                                     labels=labels,
                                     inputs=embedding_layer,
                                     num_sampled=negative_samples_n,
                                     num_classes=vocabulary_size))
optimizer = tf.train.AdagradOptimizer(0.1).minimize(loss)

In [16]:
steps_n = 15000 #len(unique_words) * 3
trained_embeddings = [] # we want to use them later
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(steps_n):
        batch_inputs, batch_labels = skipgrams_batch(batch_size)
        if i+1 == steps_n:
            _, loss_val, trained_embeddings = sess.run([optimizer, loss, embeddings], feed_dict={inputs: batch_inputs,
                                                             labels: batch_labels})
            print('Final loss:', loss_val)
        else:
            _, loss_val = sess.run([optimizer, loss], feed_dict={inputs: batch_inputs,
                                                             labels: batch_labels})
            # TODO meaningful completion info
            if (i % 10000 == 0):
                print(loss_val)
print(trained_embeddings.shape)

398.083
132.346
Final loss: 137.598
(705241, 70)


In [20]:
trained_embeddings[word_map['honey'], ]

array([  9.87995267e-02,   7.50405669e-01,  -3.45112562e-01,
         4.16200280e-01,   3.97911966e-01,  -8.51840615e-01,
         1.10917020e+00,   4.26351130e-01,   9.82077792e-02,
        -8.72896552e-01,  -7.05359355e-02,   6.93705916e-01,
        -3.22184175e-01,   4.01414007e-01,  -3.21964175e-01,
        -2.20627397e-01,  -4.07446831e-01,  -5.31882942e-01,
        -5.82467496e-01,   9.64882553e-01,  -9.04482603e-01,
        -1.72290668e-01,   1.50034636e-01,  -6.43440843e-01,
        -2.64710099e-01,  -8.46308947e-01,  -2.09136419e-02,
        -2.10848391e-01,   9.43276763e-01,  -1.55879229e-01,
         5.97461402e-01,   8.06708753e-01,   6.64570555e-02,
         4.26887095e-01,  -6.75049663e-01,   8.89676213e-01,
        -6.56679630e-01,   1.99052677e-01,   8.46626937e-01,
        -2.64855206e-01,  -4.92919147e-01,  -8.90498281e-01,
        -5.70992649e-01,   6.18981481e-01,   4.09525901e-01,
         1.24269322e-01,   5.74584585e-04,   1.13344885e-01,
         6.54852509e-01,

## Learning the transformation matrix

In [17]:
import numpy as np
from scipy.optimize import fmin_l_bfgs_b as l_bfgs

In [18]:
encoding_matrix = np.random.randn(embedding_size*2, embedding_size)
encoding_bias = np.zeros((1, embedding_size))
decoding_matrix = np.random.randn(embedding_size, embedding_size*2)
decoding_bias = np.zeros((1, embedding_size*2))

def encode_node(child1, child2):
    """Both child1 and child2 are numpy arrays of shape (1, embedding_size)"""
    return np.tanh(np.dot(np.concatenate((child1, child2)), encoding_matrix) + encoding_bias)

def decode_node(node):
    return np.tanh(np.dot(node, decoding_matrix) + decoding_bias)

In [ ]:
encoding_train_batch_size = 5 # number of sentences

def encoding_train_batch():
    sentence = [] # TODO
    
    p = Popen(['java', '-jar', 'easyccg-0.2/easyccg.jar', '--model', 'easyccg-0.2/model'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
    parse_output = p.communicate(input=' '.join(sentence).encode())[0]
    p.terminate()
    
    # Encode the tree.
    tree = ParentedTree.fromstring(clean_parse_output(parse_output))
    node_encodings = dict()
    for leaf in tree:
        node_encodings[leaf] = trained_embeddings[word_map[leaf.label()], ] # TODO
    nodes_to_visit = tree.leaves()
    while True:
        current_node = nodes_to_visit.pop()
        if not current_node.parent(): # root of the tree
            break
        left_sibling = current_node.left_sibling()
        right_sibling = current_node.right_sibling()
        if left_sibling:
            node_encodings[current_node.parent()] = encode_node(node_encodings[left_sibling], node_encodings[current_node])
        else:
            node_encodings[current_node.parent()] = encode_node(node_encodings[current_node], node_encodings[right_sibling])
        nodes_to_visit.append(current_node.parent())
        
    # Decode the tree back again.
    nodes_to_visit = [ tree.root() ]
    # this dictionary in fact maps nodes to their *partial* decodings from which their children are to be
    # recreated; thus for the root it's just its encoding, from which we will retrieve immediate children
    node_decodings = dict()
    node_decodings[tree.root()] = node_encodings[tree.root()]
    encoding_errors = dict()
    while True:
        current_node = nodes_to_visit.pop()
        children = [child for child in current_node]
        if len(children) > 0: # not a leaf
            decoded_node = decode_node(node_decodings[current_node])
            encoding_errors[current_node] = node_encodings[current_node] - node_decodings[current_node]
            node_decodings[children[0]] = decoded_node[, :embedding_size]
            node_decodings[children[1]] = decoded_node[, embedding_size:]
            
    # Compute the error value and its gradient.
    error = reduce(lambda err_sum, node_err: err_sum + node_err, encoding_errors.values(), np.zeros((1, embedding_size*2)))
    error = error / len(encoding_errors)
    # TODO gradient